In [1]:
import os
import glob
from collections import OrderedDict,defaultdict
import pipes
import pprint
import tempfile
import re

In [2]:
root_dir = 'bl6_rep1_data'

In [3]:
####Fixed length 1e6 = 100 blocks. 
####Fixed length 5e5 = 200 blocks. 
Fixed_length = 2e6

In [4]:
def binary_locate(array,ele,start,end):
    middle = (end+start)//2
    if(end-start>1):
        if(array[middle]<ele):
            return binary_locate(array,ele,middle,end)
        elif(array[middle+1]>ele):
            return binary_locate(array,ele,start,middle)
        else:
            return middle,middle+1
    else:
        return middle,middle+1

In [5]:
p = pipes.Template()
p.append("zcat", '--')
Gene_Loc = defaultdict(list)
#f = p.open('/home/longy/cnda/ensembl/Homo_sapiens.GRCh38.102.gtf.gz', 'r')

f = p.open('/home/longy/cnda/ensembl/Mus_musculus.GRCm38.102.gtf.gz', 'r')
for line in f.readlines():
    if re.match('^#',line):
        continue
    line = line.rstrip('\n')
    data = line.split('\t')
    if(data[2] == 'gene'):
        Gene_Loc['chr'+data[0]+data[6]].append(int(data[3]))
        Gene_Loc['chr'+data[0]+data[6]].append(int(data[4]))

In [6]:
import sys
sys.setrecursionlimit(10000)
array = Gene_Loc['chr1+']
binary_locate(array,46104790,0,3505)

(678, 679)

In [7]:
total = 0
files = glob.glob(root_dir+"/*.txt")
info = open(root_dir+'/Blocks/info.txt','w')
info.write('ID\tstart\tend\tlength\tsites_num\n')
for file in files:
    num_lines = sum(1 for line in open(file))
    num_blocks = round(num_lines/Fixed_length)
    len_blocks = num_lines/num_blocks
    
    f= open(file,'r')
    pre_pos = 0
    dict_line = OrderedDict()
    #dict_line = dict()
    count = 0
    separate_num=0
    save_block = 0
    start = 0
    
    for i, line in enumerate(f):
        line = line.rstrip('\n')
        if(i==0):
            continue ###Skip header
        chromosome,pos,strand = line.split('\t')[0].split(':')
        pos = int(pos)
        count += 1
        if(i==1):
            start=pos
        if(count>len_blocks and pos-pre_pos>1000 and save_block<num_blocks-1):
            array = Gene_Loc[chromosome+strand]
            index1,index2 = binary_locate(array,pre_pos,0,len(array)-1)
            if(index1%2==1 and index2%2==0):
                ww = open(root_dir+'/Blocks/%s_%s_%d'%(chromosome,strand,save_block),'w')
                print('writing file '+root_dir+'/Blocks/%s_%s_%d_%d-%d'%(chromosome,strand,save_block,start,pre_pos))
                info.write('%s_%s_%d\t%d\t%d\t%d\t%d\n'%(chromosome,strand,save_block,start,pre_pos,pre_pos-start,count))
                #print('%s_%s_%d\t%d\t%d\t%d\t%d\n'%(chromosome,strand,save_block,start,pre_pos,pre_pos-start,count))
                for key,val in dict_line.items():
                    ww.write('%s\n'%val)
                ww.close()
                dict_line.clear()
                count = 0
                save_block += 1
                start  = pos
            
        
        dict_line[pos] = line
        pre_pos = pos
        
    out = open(root_dir+'/Blocks/%s_%s_%d'%(chromosome,strand,save_block),'w')
    print('writing file '+root_dir+'/Blocks/%s_%s_%d_%d-%d'%(chromosome,strand,save_block,start,pre_pos))
    info.write('%s_%s_%d\t%d\t%d\t%d\t%d\n'%(chromosome,strand,save_block,start,pre_pos,pre_pos-start,count))
    for key,val in dict_line.items():
        out.write('%s\n'%(val))
    out.close()

info.close()

writing file bl6_rep1_data/Blocks/chr15_+_0_3053635-35603423
writing file bl6_rep1_data/Blocks/chr15_+_1_35606118-75032226
writing file bl6_rep1_data/Blocks/chr15_+_2_75034254-86498526
writing file bl6_rep1_data/Blocks/chr15_+_3_86499996-103828558
writing file bl6_rep1_data/Blocks/chr15_-_0_3167231-36333508
writing file bl6_rep1_data/Blocks/chr15_-_1_36342045-64383772
writing file bl6_rep1_data/Blocks/chr15_-_2_64391788-79403286
writing file bl6_rep1_data/Blocks/chr15_-_3_79413363-94543990
writing file bl6_rep1_data/Blocks/chr15_-_4_94547865-103901353
writing file bl6_rep1_data/Blocks/chr11_+_0_3123506-23502210
writing file bl6_rep1_data/Blocks/chr11_+_1_23503978-50343734
writing file bl6_rep1_data/Blocks/chr11_+_2_50345924-62181461
writing file bl6_rep1_data/Blocks/chr11_+_3_62182836-75164471
writing file bl6_rep1_data/Blocks/chr11_+_4_75165804-84296803
writing file bl6_rep1_data/Blocks/chr11_+_5_84297976-98925398
writing file bl6_rep1_data/Blocks/chr11_+_6_98930820-115433723
writing 

writing file bl6_rep1_data/Blocks/chr4_-_1_44243188-82790716
writing file bl6_rep1_data/Blocks/chr4_-_2_82792041-115238505
writing file bl6_rep1_data/Blocks/chr4_-_3_115239853-129672781
writing file bl6_rep1_data/Blocks/chr4_-_4_129674844-141539050
writing file bl6_rep1_data/Blocks/chr4_-_5_141576043-156252068
writing file bl6_rep1_data/Blocks/chr13_+_0_3043636-37955500
writing file bl6_rep1_data/Blocks/chr13_+_1_37957070-63881468
writing file bl6_rep1_data/Blocks/chr13_+_2_63882473-96419003
writing file bl6_rep1_data/Blocks/chr13_+_3_96420255-120304378
writing file bl6_rep1_data/Blocks/chr14_-_0_3051637-32342956
writing file bl6_rep1_data/Blocks/chr14_-_1_32344889-57828195
writing file bl6_rep1_data/Blocks/chr14_-_2_57829799-74549952
writing file bl6_rep1_data/Blocks/chr14_-_3_74607916-124689347
writing file bl6_rep1_data/Blocks/chr17_+_0_3055606-15775699
writing file bl6_rep1_data/Blocks/chr17_+_1_15797501-28333691
writing file bl6_rep1_data/Blocks/chr17_+_2_28334956-46422949
writing